<a href="https://colab.research.google.com/github/cttwjh/HITCS/blob/main/%E6%95%B0%E6%8D%AE%E9%A2%84%E5%A4%84%E7%90%86%E6%96%87%E4%BB%B6_add_keshihua_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip  install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.7 MB/s eta 0:00:00


In [ ]:
import os
import re
import random
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from itertools import product
# 第一步:筛除长度小于200个碱基的转录本序列
def filter_sequences(input_file, output_file, min_length=200):
    """
    筛除长度小于min_length的转录本序列,并保存到output_file中
    """
    with open(output_file, "w") as out_handle:
        for record in SeqIO.parse(input_file, "fasta"):
            if len(record.seq) >= min_length:
                SeqIO.write(record, out_handle, "fasta")
# 第二步:对mRNA序列进行筛选和预处理
def preprocess_mrna(input_file, output_file):
    """
    选出注释有"mRNA"的转录本序列,删除注释有"putative","predicted","pseudogene"的mRNA转录本序列
    将碱基"U"全部替换为"T",并将混合碱基全部替换为未知碱基"N"
    """
    with open(output_file, "w") as out_handle:
        for record in SeqIO.parse(input_file, "fasta"):
            if "mRNA" in record.description and not any(
                keyword in record.description
                for keyword in ["putative", "predicted", "pseudogene"]
            ):
                record.seq = str(record.seq).upper().replace("U", "T")
                record.seq = re.sub(r"[^ATCGN]", "N", str(record.seq))
                SeqIO.write(record, out_handle, "fasta")
# 第三步:为lncRNA和mRNA序列添加标签
def add_labels(lncrna_file, mrna_file, output_file):
    """
    将lncRNA转录本序列视为正类样本集,用标签"0"来表示;mRNA转录本序列为负类样本集,用标签"1"来表示
    将两个数据集合并并保存到output_file中
    """
    with open(output_file, "w") as out_handle:
        for record in SeqIO.parse(lncrna_file, "fasta"):
            record.description += " label=0"
            SeqIO.write(record, out_handle, "fasta")
        for record in SeqIO.parse(mrna_file, "fasta"):
            record.description += " label=1"
            SeqIO.write(record, out_handle, "fasta")
# 可选:对lncRNA和mRNA序列进行下采样,使两者数量平衡
def downsample_sequences(input_file, output_file):
    """
    对lncRNA和mRNA序列进行下采样,使两者数量平衡
    """
    lncrna_records = []
    mrna_records = []
    for record in SeqIO.parse(input_file, "fasta"):
        if "label=0" in record.description:
            lncrna_records.append(record)
        else:
            mrna_records.append(record)
    min_count = min(len(lncrna_records), len(mrna_records))
    lncrna_records = random.sample(lncrna_records, min_count)
    mrna_records = random.sample(mrna_records, min_count)
    with open(output_file, "w") as out_handle:
        for record in lncrna_records + mrna_records:
            SeqIO.write(record, out_handle, "fasta")
# 可选:对平衡后的数据集进行抽样,抽取10%的数据
def subsample_sequences(input_file, output_file, subsample_ratio=0.1):
    """
    对平衡后的数据集进行抽样,抽取subsample_ratio的数据
    """
    records = list(SeqIO.parse(input_file, "fasta"))
    subsample_count = int(len(records) * subsample_ratio)
    subsample_records = random.sample(records, subsample_count)
    with open(output_file, "w") as out_handle:
        for record in subsample_records:
            SeqIO.write(record, out_handle, "fasta")

# 第四步:计算k-mer使用频率和ORF长度
def calculate_features(input_file, k_values=[1, 2, 3, 4, 5, 6]):
    """
    计算每条转录本序列的k-mer使用频率和ORF长度,并构建特征向量
    返回特征矩阵X和标签向量y,其中k-mer频率的特征向量大小为1×5460,ORF长度特征向量大小为1×1,总的输入特征向量大小为1×5461
    """
    X = []
    y = []
    orf_lengths = []
    # 生成所有可能的k-mer
    kmers = []
    for k in k_values:
        kmers.extend(["".join(kmer) for kmer in product("ATCG", repeat=k)])
    for record in SeqIO.parse(input_file, "fasta"):
        sequence = str(record.seq)
        seq_length = len(sequence)
        # 计算k-mer使用频率
        k_mer_freqs = []
        for kmer in kmers:
            k = len(kmer)
            count = sequence.count(kmer)
            w_k = 1 / (4 ** (6 - k))
            freq = count / (seq_length - k + 1) * w_k
            k_mer_freqs.append(freq)
        # 计算ORF长度
        start_codons = ["ATG"]
        stop_codons = ["TAA", "TAG", "TGA"]
        max_orf_length = 0
        for frame in range(3):
            start_positions = []
            for i in range(frame, seq_length, 3):
                codon = sequence[i : i + 3]
                if codon in start_codons:
                    start_positions.append(i)
            for start_pos in start_positions:
                for i in range(start_pos, seq_length, 3):
                    codon = sequence[i : i + 3]
                    if codon in stop_codons:
                        orf_length = i - start_pos + 3
                        max_orf_length = max(max_orf_length, orf_length)
                        break
        orf_lengths.append(max_orf_length)
        max_orf_length = np.log10(max_orf_length) if max_orf_length > 0 else 0
        # 特征向量
        features = k_mer_freqs + [max_orf_length]
        X.append(features)
        # 标签
        if "label=0" in record.description:
            y.append(0)
        else:
            y.append(1)
    # 归一化ORF长度
    orf_lengths = np.log10(np.array(orf_lengths) + 1)  # 加1避免log10(0)
    orf_lengths = (orf_lengths - orf_lengths.min()) / (orf_lengths.max() - orf_lengths.min() + 1e-8)  # 加1e-8避免除以0
    X = np.array(X)
    X[:, -1] = orf_lengths
    return X, np.array(y)




# 可视化

In [ ]:
# 老的可视化:绘制ORF长度和k-mer使用频率分布图
def visualize_features(X, y, k_values=[1, 2, 3, 4, 5, 6]):
    """
    可视化ORF长度和k-mer使用频率分布图
    """
    plt.figure(figsize=(12, 4))
    # 绘制ORF长度分布图
    plt.subplot(1, 2, 1)
    plt.hist(X[y == 0, -1], bins=50, alpha=0.5, label="lncRNA", color="blue")
    plt.hist(X[y == 1, -1], bins=50, alpha=0.5, label="mRNA", color="red")
    plt.xlabel("Normalized ORF Length")
    plt.ylabel("Frequency")
    plt.legend()
    plt.title("ORF Length Distribution")
    # 绘制k-mer使用频率分布图
    plt.subplot(1, 2, 2)
    for i, k in enumerate(k_values):
        start = sum(4 ** j for j in range(k - 1))
        end = start + 4 ** (k - 1)
        plt.hist(X[y == 0, start:end].mean(axis=1), bins=50, alpha=0.5, label=f"lncRNA (k={k})", color="blue")
        plt.hist(X[y == 1, start:end].mean(axis=1), bins=50, alpha=0.5, label=f"mRNA (k={k})", color="red")
    plt.xlabel("k-mer Frequency")
    plt.ylabel("Frequency")
    plt.legend()
    plt.title("k-mer Frequency Distribution")
    plt.tight_layout()
    plt.show()

import seaborn as sns
import matplotlib.pyplot as plt

def visualize_features(X, y):
    # 将特征矩阵X拆分为k-mer使用频率和ORF长度两个特征
    k_mer_freqs = X[:, :-1]
    orf_lengths = X[:, -1]

    # 创建一个DataFrame,包含特征和标签信息
    df = pd.DataFrame({
        'k-mer_frequency': k_mer_freqs.mean(axis=1),
        'ORF_length': orf_lengths,
        'RNA_type': ['lncRNA' if label == 0 else 'mRNA' for label in y]
    })

    # 绘制箱线图
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    sns.boxplot(x='RNA_type', y='k-mer_frequency', data=df, ax=axes[0])
    sns.boxplot(x='RNA_type', y='ORF_length', data=df, ax=axes[1])
    axes[0].set_title('K-mer Frequency')
    axes[1].set_title('ORF Length')
    plt.tight_layout()
    plt.show()

    # 绘制小提琴图
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    sns.violinplot(x='RNA_type', y='k-mer_frequency', data=df, ax=axes[0])
    sns.violinplot(x='RNA_type', y='ORF_length', data=df, ax=axes[1])
    axes[0].set_title('K-mer Frequency')
    axes[1].set_title('ORF Length')
    plt.tight_layout()
    plt.show()

    # 绘制散点图
    sns.scatterplot(x='k-mer_frequency', y='ORF_length', hue='RNA_type', data=df)
    plt.title('K-mer Frequency vs. ORF Length')
    plt.show()

    # 绘制相关性图
    sns.jointplot(x='k-mer_frequency', y='ORF_length', kind='scatter', hue='RNA_type', data=df)
    plt.show()



# 第六步:保存特征和标签到文件
def save_features(X, y, output_file):
    """
    将特征X和标签y保存到output_file中
    """
    np.savez(output_file, X=X, y=y)
# 主函数
def main():
    # 输入文件路径
    lncrna_file = "/content/drive/MyDrive/gencode.v38.lncRNA_transcripts.faa"
    mrna_file = "/content/drive/MyDrive/human.14.rna.fna"
    # 中间文件路径
    filtered_lncrna_file = "filtered_lncrna.fasta"
    filtered_mrna_file = "filtered_mrna.fasta"
    preprocessed_mrna_file = "preprocessed_mrna.fasta"
    labeled_file = "labeled_sequences.fasta"
    balanced_file = "balanced_sequences.fasta"
    subsampled_file = "subsampled_sequences.fasta"
    # 输出文件路径
    output_file = "features.npz"
    # 第一步:筛除长度小于200个碱基的转录本序列
    filter_sequences(lncrna_file, filtered_lncrna_file)
    filter_sequences(mrna_file, filtered_mrna_file)
    # 第二步:对mRNA序列进行筛选和预处理
    preprocess_mrna(filtered_mrna_file, preprocessed_mrna_file)
    # 第三步:为lncRNA和mRNA序列添加标签
    add_labels(filtered_lncrna_file, preprocessed_mrna_file, labeled_file)
    # 可选:对lncRNA和mRNA序列进行下采样,使两者数量平衡
    downsample_sequences(labeled_file, balanced_file)
    # 可选:对平衡后的数据集进行抽样,抽取10%的数据
    subsample_sequences(balanced_file, subsampled_file)
    # 第四步:计算k-mer使用频率和ORF长度
    X, y = calculate_features(subsampled_file)
    # 可视化:绘制ORF长度和k-mer使用频率分布图
    visualize_features(X, y)
    # 第六步:保存特征和标签到文件
    save_features(X, y, output_file)
if __name__ == "__main__":
    main()

# 函数

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/gencode.v38.lncRNA_transcripts.faa'